In [226]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [227]:
# 요약 함수 모음

## train, test 불러오기
def load_traintest_dt():
    train = pd.read_csv('../data/titanic/train.csv')
    test = pd.read_csv('../data/titanic/test.csv')  

    return (train, test)

## submission_a 불러오기
def load_submission_a() :
    submission_a = pd.read_csv('../data/submission_a.csv')
    del submission_a['PassengerId']

    return submission_a

## 피처 삭제
def drop_col(train,test,col_list):
    '''
    drop train, test column
    parameter : train,test, list
    return train,test
    '''
    train = train.drop(col_list,axis=1)
    test = test.drop(col_list, axis=1)
    return (train,test)

## train, target 나누기
def split_data(train):
    train_data = train.drop('Survived',axis=1)
    target = train['Survived']

    return (train_data, target)

In [228]:
# 데이터 전처리 함수

    # Name,Title
def Name_cleaning(train,test):
    
    train_test_data = [train,test]
    for dataset in train_test_data:
        dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.',expand=False)

    train_title_mapping = {'Mr':0, 'Miss':1, 'Mrs':2, 'Master':3
                    , 'Dr':4, 'Rev':4, 'Mlle':4, 'Major':4, 'Col':4
                    ,'Countess':4, 'Capt':4, 'Ms':4, 'Sir':4, 'Lady':4
                    , 'Mme':4, 'Don':4, 'Jonkheer':4
                    }
    train['Title'] = train['Title'].map(train_title_mapping)

    test_title_mapping = {'Mr':0, 'Miss':1, 'Mrs':2, 'Master':3
                    , 'Col':4, 'Rev':4, 'Ms':4, 'Dr':4, 'Dona':4
                    }

    test['Title'] = test['Title'].map(test_title_mapping)

    return (train,test)

    # sex
def sex_cleaning(train,test):
    sex_mapping = {'male':0, 'female':1}
    train_test_data = [train,test]
    for dataset in train_test_data:
        dataset['Sex'] = dataset['Sex'].map(sex_mapping)

    return (train,test)

    # Have Age
def haveage_cleaning(train,test):

    train.loc[train['Age'].isnull(), 'Null_Age'] = 0
    test.loc[test['Age'].isnull(), 'Null_Age'] = 0

    train.loc[train['Age'].notnull(), 'Null_Age'] = 1
    test.loc[test['Age'].notnull(), 'Null_Age'] = 1

    return (train,test)


    # Age
def age_cleaning(train,test):
    
    train['Age'].fillna(train.groupby('Title')['Age'].transform('mean'),inplace=True)
    test['Age'].fillna(test.groupby('Title')['Age'].transform('mean'),inplace=True)

    train_test_data = [train,test]
    for dataset in train_test_data:
        dataset.loc[dataset['Age'] <= 17, 'Age'] =0
        dataset.loc[(dataset['Age'] >17) & (dataset['Age'] <= 24), 'Age'] = 1
        dataset.loc[(dataset['Age'] >24) & (dataset['Age'] <= 34), 'Age'] = 2
        dataset.loc[(dataset['Age'] >34) & (dataset['Age'] <= 44), 'Age'] = 3
        dataset.loc[(dataset['Age'] >44) & (dataset['Age'] <= 60), 'Age'] = 4
        dataset.loc[dataset['Age'] >60, 'Age'] = 5

    return (train,test)


    # Embarked
def embarked_cleaning(train,test):

    train['Embarked'] = train['Embarked'].fillna('S')
    test['Embarked'] = test['Embarked'].fillna('S')

    embarked_mapping = {'S':0, 'C':1, 'Q':2}
    train['Embarked'] = train['Embarked'].map(embarked_mapping)
    test['Embarked'] = test['Embarked'].map(embarked_mapping)

    return (train,test)

    #Fare
def fare_cleaning(train,test):

    test['Fare'].fillna(
        test.groupby('Pclass')['Fare'].transform('median'), inplace=True
    )

    return (train,test)

    # group_size
def groupsize_cleanig(train,test):

    for ticket_num in train['Ticket'].unique():
        train.loc[train['Ticket']==ticket_num,'group_size'] = len(train[train['Ticket']==ticket_num])

    for ticket_num in test['Ticket'].unique():
        test.loc[test['Ticket']==ticket_num,'group_size'] = len(test[test['Ticket']==ticket_num])

    train['Fare'] = train['Fare']/train['group_size']
    test['Fare'] = test['Fare']/test['group_size']
    
    train_test_data = [train,test]
    for dataset in train_test_data:
        dataset.loc[dataset['group_size'] == 1, 'group_size'] = 0
        dataset.loc[dataset['group_size'] == 2, 'group_size'] = 0.4
        dataset.loc[(dataset['group_size'] == 3) | (dataset['group_size'] == 4), 'group_size'] = 0.8
        dataset.loc[dataset['group_size'] > 4, 'group_size'] = 1.2

    train_test_data = [train,test]
    for dataset in train_test_data:

        dataset.loc[dataset['Fare'] <= 7,'Fare'] = 0
        dataset.loc[(dataset['Fare'] >7) & (dataset['Fare'] <=8.8), 'Fare'] = 0.4
        dataset.loc[(dataset['Fare'] >8.8) & (dataset['Fare'] <=17), 'Fare'] = 0.8
        dataset.loc[(dataset['Fare'] >17) & (dataset['Fare'] <=30), 'Fare'] = 1.2
        dataset.loc[(dataset['Fare'] >30) & (dataset['Fare'] <=100), 'Fare'] = 1.6
        dataset.loc[dataset['Fare'] > 100,'Fare'] = 2
        
    return (train,test)

    # Have Cabin
def havecabin_cleaning(train,test):

    train.loc[train['Cabin'].isnull(), 'Null_Cabin'] = 0
    test.loc[test['Cabin'].isnull(), 'Null_Cabin'] = 0

    train.loc[train['Cabin'].notnull(), 'Null_Cabin'] = 1
    test.loc[test['Cabin'].notnull(), 'Null_Cabin'] = 1

    return (train,test)

    # Cabin
def cabin_cleaning(train,test):

    train['Cabin'] = train['Cabin'].str[:1]
    test['Cabin'] = test['Cabin'].str[:1]

    cabin_mapping = {"A": 0, "B": 0.4
                    , "C": 0.8, "D": 1.2
                    , "E": 1.6, "F": 2, "G": 2.4
                    , 'T' :2.8
                    }

    train['Cabin'] = train['Cabin'].map(cabin_mapping)
    test['Cabin'] = test['Cabin'].map(cabin_mapping)

    train["Cabin"].fillna(train.groupby("Pclass")["Cabin"].transform("median"), inplace=True)
    test["Cabin"].fillna(test.groupby("Pclass")["Cabin"].transform("median"), inplace=True)

    return (train,test)

    # FamilySize
def familysize_cleaning(train,test):

    train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
    test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

    family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
    train['FamilySize'] = train['FamilySize'].map(family_mapping)
    test['FamilySize'] = test['FamilySize'].map(family_mapping)

    return (train,test)

    # 정규화
def data_scaler(train,test):

    columns = ['Pclass', 'Sex', 'Age', 'Fare', 'Cabin',
        'Embarked', 'Title', 'Null_Age', 'group_size', 'Null_Cabin',
        'FamilySize','Family_Survival']
 
    train_test_data = [train,test]
    for dataset in train_test_data:
        scaler = MinMaxScaler()
        # scaler = StandardScaler()

        scaler.fit(dataset[columns])
        scaled = scaler.transform(dataset[columns])

        df_scaled = pd.DataFrame(data=scaled, columns=columns)
        dataset.loc[:,columns] = df_scaled

    return (train,test)

In [229]:
# 전처리 전체 실행

def data_cleaning():
    train,test = load_traintest_dt()

    #name
    train,test = Name_cleaning(train,test)
    #sex
    train,test = sex_cleaning(train,test)
    #groupsize
    train,test = groupsize_cleanig(train,test)
    #haveage
    train,test = haveage_cleaning(train,test)
    #age
    train,test = age_cleaning(train,test)
    #embarked
    train,test = embarked_cleaning(train,test)
    #fare
    train,test = fare_cleaning(train,test)
    #havecabin
    train,test = havecabin_cleaning(train,test)
    #cabin
    train,test = cabin_cleaning(train,test)
    #familysize
    train,test = familysize_cleaning(train,test)
    
    #drop
    train,test = drop_col(train,test,['Ticket','SibSp','Parch','Name'])
    
    #scaler
    train,test = data_scaler(train,test)


    #split

    train_data, target = split_data(train)

    return (train_data,target,test)

In [230]:
train,test = load_traintest_dt()

#name
train,test = Name_cleaning(train,test)
#sex
train,test = sex_cleaning(train,test)
#haveage
train,test = haveage_cleaning(train,test)
#age
train,test = age_cleaning(train,test)
#embarked
train,test = embarked_cleaning(train,test)
#fare
train,test = fare_cleaning(train,test)
#groupsize
train,test = groupsize_cleanig(train,test)
#havecabin
train,test = havecabin_cleaning(train,test)
#cabin
train,test = cabin_cleaning(train,test)
#familysize
train,test = familysize_cleaning(train,test)

In [231]:
data_df = pd.concat([train,test])

data_df['Lastname'] = data_df['Name'].apply(lambda x: str.split(x, ",")[0])
# data_df['Fare'].fillna(data_df['Fare'].mean(), inplace=True)

DEFAULT_SURVIVAL_VALUE = 0.5
data_df['Family_Survival'] = DEFAULT_SURVIVAL_VALUE

for grp, grp_df in data_df[['Survived','Name', 'Lastname', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['Lastname', 'Fare']):
    
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0):
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0

print("Number of passengers with family survival information:", 
      data_df.loc[data_df['Family_Survival']!=0.5].shape[0])

Number of passengers with family survival information: 420


In [232]:
for _, grp_df in data_df.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0):
                    data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0
                        
print("Number of passenger with family/group survival information: " 
      +str(data_df[data_df['Family_Survival']!=0.5].shape[0]))

# # Family_Survival in TRAIN_DF and TEST_DF:
train['Family_Survival'] = data_df['Family_Survival'][:891]
test['Family_Survival'] = data_df['Family_Survival'][891:]

Number of passenger with family/group survival information: 594


In [233]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Null_Age,group_size,Null_Cabin,FamilySize,Family_Survival
0,1,0,3,"Braund, Mr. Owen Harris",0,1.0,1,0,A/5 21171,0.4,2.0,0,0,1.0,0.0,0.0,0.4,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,3.0,1,0,PC 17599,1.6,0.8,1,2,1.0,0.0,1.0,0.4,0.5
2,3,1,3,"Heikkinen, Miss. Laina",1,2.0,0,0,STON/O2. 3101282,0.4,2.0,0,1,1.0,0.0,0.0,0.0,0.5


In [234]:
#drop
train,test = drop_col(train,test,['Ticket','SibSp','Parch','Name'])

In [235]:
train,test = data_scaler(train,test)

In [236]:
# train.describe()

In [237]:
train_data, target = split_data(train)

In [238]:
submission_a = load_submission_a()

In [239]:
# 0.8277511961722488
clf = RandomForestClassifier(n_estimators=110
                             , max_depth=4
                             #, min_samples_split=6
                             , min_samples_leaf=30
                             , random_state=177
                             ,n_jobs=-1)
clf.fit(train_data, target)

prediction = clf.predict(test)

accuracy = accuracy_score(prediction, submission_a)

accuracy

0.8277511961722488

In [240]:
submission = pd.DataFrame({

    'PassengerId':test['PassengerId']
    ,'Survived':prediction
    
})
submission.to_csv('submission.csv',index=False)

In [ ]:
acc_list=[]

for r in range(500):
    clf = RandomForestClassifier(max_depth=4
                              ,n_estimators=300
                             ,min_samples_leaf=32
                             , random_state=r      # 177 827
                                 , n_jobs=-1)
    clf.fit(train_data, target) # 학습
    Y_pred = clf.predict(test) # 테스트 데이터로 예측값 추출    
    
    accuracy = accuracy_score(Y_pred, submission_a)
    print(r, accuracy)
    acc_list.append(accuracy)

acc_list.index(max(acc_list)),max(acc_list)

In [ ]:

dt_reg = DecisionTreeRegressor(random_state=0, max_depth=4)
rf_reg = RandomForestRegressor(random_state=0, n_estimators=1000)
gb_reg = GradientBoostingRegressor(random_state=0, n_estimators=1000)
xgb_reg = XGBRegressor(n_estimators=1000)
lgb_reg = LGBMRegressor(n_estimators=1000)

# 트리 기반의 회귀 모델을 반복하면서 평가 수행 
models = [dt_reg, rf_reg, gb_reg, xgb_reg, lgb_reg]
for model in models:  
    get_model_cv_prediction(model, train_data, target)
